In [45]:
from EncoderDecoderShareVAE import EncoderDecoderShareVAE
import NewsVAEArguments
from transformers import RobertaTokenizer
import torch
from NewsData import NewsData
import numpy as np
from functools import partial
from datasets import load_dataset
from torch.utils.data import DataLoader

# Check if on server
if not torch.cuda.is_available():
    print("Start Jupyter server on LISA pls.")

Start Jupyter server on LISA pls.


In [46]:
loaders = {}

# Get data
data = NewsData(args.dataset_name, args.tokenizer_name,
                batch_size=args.batch_size, num_workers=args.num_workers,
                pin_memory=True, debug=args.debug_data,
                debug_data_len=args.debug_data_len, max_seq_len=args.max_seq_len)

In [47]:
args.ddp = True
for phase in ['train', 'validation']:
    if args.ddp:
        sampler = torch.utils.data.distributed.DistributedSampler(data.datasets[phase], rank=0,
                                                                  num_replicas=1, shuffle=True)
        # With distributed sampling, shuffle must be false
        loaders[phase] = DataLoader(data.datasets[phase], batch_size=args.batch_size,
                                    sampler=sampler, shuffle=False, pin_memory=False,
                                    num_workers=args.num_workers, collate_fn=data.collate_fn)
    else:
        # Without distributed sampling
        loaders[phase] = DataLoader(data.datasets[phase], batch_size=args.batch_size,
                                    shuffle=True, pin_memory=True,
                                    num_workers=args.num_workers, collate_fn=data.collate_fn)

In [51]:
for x in range(5):
    print("--> round", x)
    print(len(loaders['train']))
#     for batch_i, batch in enumerate(loaders['train']):
# #         if batch_i == 3:
# #             break
#         print(batch['input_ids'])

In [2]:
args = NewsVAEArguments.preprare_parser(jupyter=True)
args.ddp = False
args.n_gpus = 0

----------------------------------------------------------------------------------------------------
SOME IMPORTANT ARGUMENTS
----------------------------------------------------------------------------------------------------
MAX SEQ LEN: 64
OBJECTIVE: beta-vae
BETA-VAE OBJECTIVE with KL ANNEALING:
KL-ANNEALING (step per effective batch size): True
HINGE TARGET KL: 0.5
------------------------------
N_GPUS: 0
DDP: False
BATCH SIZE: 32
GRADIENT ACCUMULATION (N BATCHES): 1
EFFECTIVE BATCHSIZE PER GRAD STEP: 0
------------------------------
LR: 0.05
TIE WEIGHTS: True
GRAD CHECKPOINT: False
------------------------------
CHECKPOINTING: False
LOGGING: True
RUN PREFIX: TEST
------------------------------
TRAIN (GLOBAL) STEPS: 50000
TRAIN EPOCH LEN: 2000
VALID EPOCH LEN: 100
----------------------------------------------------------------------------------------------------


In [3]:
VAE_model = EncoderDecoderShareVAE(args, args.base_checkpoint_name, do_tie_weights=args.do_tie_weights)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VAE_Decoder_RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<class 'VAE_Decoder_Roberta.VAE_Decoder_RobertaModel'> and <class 'transformers.modeling_roberta.RobertaModel'> are not equal. In this case make sure that all encoder weights are correctly initialized. 
The following encoder weights were not tied to the decoder ['roberta/pooler']


In [4]:
def convert_to_features(data_batch, tokenizer=None, args=None):
    encoded_batch = tokenizer(data_batch['article'], truncation=True, max_length=args.max_seq_len, padding=True)
    encoded_batch['article'] = [a for a in data_batch['article']]
    encoded_batch['id'] = [i for i in data_batch['id']]
    return encoded_batch

dataset_validation = load_dataset('cnn_dailymail', name='3.0.0', ignore_verifications=True, split='validation[:1000]')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
encoded_dataset_validation = dataset_validation.map(partial(convert_to_features, tokenizer=tokenizer, args=args), batched=True)

I1112 12:07:55.105442 4518768064 filelock.py:274] Lock 140544657157648 acquired on /Users/claartje/.cache/huggingface/datasets/4b801baa55da8559101ff966972c8ee2ebc8962c04d7c12172781053e957de7a.1bf1bb5abb0006b64eb0c53264061ddb85f91fa7f9deec04a4de38a4c09a0ec0.py.lock
I1112 12:07:55.236371 4518768064 filelock.py:318] Lock 140544657157648 released on /Users/claartje/.cache/huggingface/datasets/4b801baa55da8559101ff966972c8ee2ebc8962c04d7c12172781053e957de7a.1bf1bb5abb0006b64eb0c53264061ddb85f91fa7f9deec04a4de38a4c09a0ec0.py.lock
I1112 12:07:55.649162 4518768064 filelock.py:274] Lock 140544655829648 acquired on /Users/claartje/.cache/huggingface/datasets/_Users_claartje_.cache_huggingface_datasets_cnn_dailymail_3.0.0_3.0.0_0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602.lock
I1112 12:07:55.652176 4518768064 filelock.py:318] Lock 140544655829648 released on /Users/claartje/.cache/huggingface/datasets/_Users_claartje_.cache_huggingface_datasets_cnn_dailymail_3.0.0_3.0.0_012861

In [5]:
def get_random_batch_ids(low, high, size):
    random_ints = []
    while len(random_ints) != size:
        r = np.random.randint(low, high)
        if r not in random_ints:
            random_ints.append(r)
    return list(random_ints)

def get_random_batch(dataset, batchsize=32):
    ids = get_random_batch_ids(0, len(dataset), batchsize)
    attention_mask_batch = torch.LongTensor(dataset[ids]['attention_mask'])
    input_ids_batch = torch.LongTensor(dataset[ids]['input_ids'])
    input_batch = {'attention_mask': attention_mask_batch, 'input_ids':input_ids_batch}
    return dataset[ids], input_batch

batch_all, input_batch = get_random_batch(encoded_dataset_validation)

In [21]:
VAE_model.eval()

with torch.no_grad():
    predictions = VAE_model(input_ids=input_batch['input_ids'],
                            attention_mask=input_batch['attention_mask'],
                            beta=1.0, args=args, return_predictions=True)
    predictions['total_loss'] = predictions['total_loss'].item() # detach the graph
    
    # Get predicted ids
    predictions['predictions'] = torch.nn.functional.softmax(predictions['logits'].view(-1, predictions['logits'].shape[-1]), dim=-1).argmax(dim=1)
    predictions['predictions'] = predictions['predictions'].reshape(predictions['logits'].shape[0], predictions['logits'].shape[1])

In [22]:
tokenizer.batch_decode(predictions['predictions'])

['</s></s></s></s></s></s></s></s></s>/</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s> weight</s></s>,</s></s></s></s>mm</s></s></s></s></s></s></s></s></s></s>ont>> weight,</s>, weight</s>/,</s></s>,</s></s></s>',
 '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>ing</s></s></s></s></s></s> play Video</s></s> video trade</s></s></s></s></s> public video public</s></s></s></s></s></s></s></s></s></s></s></s></s>.</s></s></s> dad dad dad dad. Video</s> local</s></s></s>',
 '<s></s></s></s></s>er and and and and and and and and and and and and and and and and and and and andp andig ando and and and and and and and and and and andpand and and andp and andp andp and and and and and andp andand and and',
 '<s></s></s> uping in up</s> up up Americans up up up up jack up up over up up up plus super super super Peak up up super super up plus super super New New combined super up up New New Peak Peak super super super super up super super super super super s